<h1>Capstone Project</h1>

<h2>Stock Prediction Using Twitter Sentiment Analysis</h2>

In [2]:
'''
load the dataset DJIA dataset(June 2009 to December 2009)
'''

import os
import numpy as np
import pandas as pd
from IPython.display import display

data = open('HistoricalPrices.csv', 'rb')

df = pd.read_csv(data)
display(df.head(30))

,Date,Open,High,Low,Close
0,12/16/09,10449.81,10509.97,10427.59,10441.12
1,12/15/09,10499.31,10499.31,10426.69,10452.00
2,12/14/09,10471.28,10514.66,10471.28,10501.05
3,12/11/09,10403.41,10484.05,10400.08,10471.50
4,12/10/09,10336.00,10444.60,10335.77,10405.83
5,12/09/09,10282.85,10342.27,10235.63,10337.05
6,12/08/09,10385.42,10385.65,10249.84,10285.97
7,12/07/09,10386.86,10443.16,10360.18,10390.11
8,12/04/09,10368.57,10516.70,10311.81,10388.90
9,12/03/09,10455.63,10507.63,10350.05,10366.15


In [3]:
df.iloc[3]['Date']

'12/11/09'

In [11]:
'''
get all the tweets
'''

tweets = open('TaggedData.csv', 'rb')
thead = pd.read_csv(tweets, usecols=['Date', 'Name', 'Tweets', 'sentiment'])
display(thead.head(30))

,Date,Name,Tweets,sentiment
0,17 Jun 2009,Rodney Holston,is taking this day to relax and catch up on sl...,1
1,17 Jun 2009,Angie,went to chiro today very sore now but know i w...,0
2,17 Jun 2009,Jaxon Brooks,updated the iphone 2g to os 3 0 feels zippier ...,0
3,17 Jun 2009,K,i am sooo hyped for this weekend going to va f...,1
4,17 Jun 2009,Nicola Atkins,was feeling down earlier but two wonderful fri...,1
5,17 Jun 2009,Anonymous,mclifford973 chefdebbiecpc is absolutely corr...,1
6,17 Jun 2009,Joel Ivory Johnson,creating a reporting solution with excel feels...,1
7,17 Jun 2009,Stephanie S,rt shayminn please play the song and feel free...,0
8,17 Jun 2009,quicksilver,feeling the slump you can always swim the shar...,1
9,17 Jun 2009,Milt Fresh,a lot of local artists feel open mic nights ar...,0
